# Similarity Ensemble

In [2]:
cd ../

/Users/emanueleghelfi/Development/RS/recsys_challenge_2017


In [3]:
from src.utils.loader import *
from scipy.sparse import *
import numpy as np
from src.utils.feature_weighting import *
from src.utils.matrix_utils import compute_cosine, top_k_filtering, yadistance
from src.utils.BaseRecommender import BaseRecommender


class ContentBasedFiltering(BaseRecommender):

    """
    Good conf: tag aggr 3,10; tfidf l1 norm over all matrix
    MAP@5  0.11772497678137457 with 10 shrinkage,
                                    100 k_filtering and other as before
    MAP@5  0.12039006297936491 urm weight 0.7
    MAP@5  0.12109109578826009 without playcount and duration

    Current best:
    CBF (album 1.0, artists 1.0, no duration/playcount)
        + URM 0.8
        + TOP-55 (TFIDF (tags 1.0))
        MAP@5 0.11897304011860126
        Public leaderboard: 0.09616


    """

    def __init__(self, shrinkage=10, k_filtering=100):
        # final matrix of predictions
        self.R_hat = None

        # for keeping reference between playlist and row index
        self.pl_id_list = []
        # for keeping reference between tracks and column index
        self.tr_id_list = []

        self.shrinkage = shrinkage
        self.k_filtering = k_filtering

    def fit(self, urm, target_playlist, target_tracks, dataset, topK_tag=55):
        """
        urm: user rating matrix
        target playlist is a list of playlist id
        target_tracks is a list of track id
        shrinkage: shrinkage factor for significance weighting
        S = ICM' ICM
        R = URM S
        In between eliminate useless row of URM and useless cols of S
        """
        # initialization

        self.pl_id_list = list(target_playlist)
        self.tr_id_list = list(target_tracks)
        self.dataset = dataset
        S = None
        urm = urm.tocsr()
        print("CBF started")
        # get ICM from dataset, assume it already cleaned
        icm = dataset.build_icm()

        # Build the tag matrix, apply TFIDF
        print("Build tags matrix and apply TFIDF...")
        icm_tag = dataset.build_tags_matrix()
        #tag_weight = np.array(urm.dot(icm_tag.transpose()).sum(axis=0)).squeeze()
        #print(tag_weight[tag_weight>5].shape)
        #icm_tag = icm_tag[tag_weight>5].multiply(tag_weight[tag_weight>5])
        tags = applyTFIDF(icm_tag, norm='l1')
        #tags = icm_tag[tag_weight>5]
        # Before stacking tags with the rest of the ICM, we keep only
        # the top K tags for each item. This way we try to reduce the
        # natural noise added by such sparse features.
        tags = top_k_filtering(tags.transpose(), topK=topK_tag).transpose()
        #tags.data = tags.data / np.max(tags.data)
        #newvalue= (max'-min')/(max-min)*(value-max)+max'

        # User augmented UCM
        # print("Building User augmented ICM")
        # ucm = dataset.build_ucm()
        # ua_icm = user_augmented_icm(urm, ucm)
        # ua_icm = top_k_filtering(ua_icm.transpose(), topK=55).transpose()

        # stack all
        icm = vstack([icm, tags, urm * 1], format='csr')
        # icm = vstack([icm, tags, applyTFIDF(urm)], format='csr')

        S = compute_cosine(icm.transpose()[[dataset.get_track_index_from_id(x)
                                            for x in self.tr_id_list]],
                       icm,
                       k_filtering=self.k_filtering,
                       shrinkage=self.shrinkage,
                       n_threads=4,
                       chunksize=1000)
        s_norm = S.sum(axis=1)

        # Normalize S
        S = S.multiply(csr_matrix(np.reciprocal(s_norm)))
        print("Similarity matrix ready!")

        # Keep only the target playlists in the URM
        urm_cleaned = urm[[dataset.get_playlist_index_from_id(x)
                           for x in self.pl_id_list]]
        self.S = S.transpose()

        # Compute ratings
        R_hat = urm_cleaned.dot(S.transpose().tocsc()).tocsr()
        print("R_hat done")

        # Remove the entries in R_hat that are already present in the URM
        urm_cleaned = urm_cleaned[:, [dataset.get_track_index_from_id(x)
                                      for x in self.tr_id_list]]
        R_hat[urm_cleaned.nonzero()] = 0
        R_hat.eliminate_zeros()

        print("Shape of final matrix: ", R_hat.shape)
        self.R_hat = R_hat

    def getW(self):
        """
        Returns the similary matrix with dimensions I x I
        S is IxT
        """
        return self.S.tocsr()

    def predict(self, at=5):
        """
        returns a dictionary of
        'pl_id': ['tr_1', 'tr_at'] for each playlist in target playlist
        """
        recs = {}
        for i in range(0, self.R_hat.shape[0]):
            pl_id = self.pl_id_list[i]
            pl_row = self.R_hat.data[self.R_hat.indptr[i]:
                                     self.R_hat.indptr[i + 1]]
            # get top 5 indeces. argsort, flip and get first at-1 items
            sorted_row_idx = np.flip(pl_row.argsort(), axis=0)[0:at]
            track_cols = [self.R_hat.indices[self.R_hat.indptr[i] + x]
                          for x in sorted_row_idx]
            tracks_ids = [self.tr_id_list[x] for x in track_cols]
            recs[pl_id] = tracks_ids
        return recs

    def getR_hat(self):
        return self.R_hat

    def get_model(self):
        """
        Returns the complete R_hat
        """
        return self.R_hat.copy()


def applyTFIDF(matrix, norm='l1'):
    from sklearn.feature_extraction.text import TfidfTransformer
    transformer = TfidfTransformer(norm=norm, use_idf=True,
                                   smooth_idf=True, sublinear_tf=False)
    tfidf = transformer.fit_transform(matrix.transpose())
    return tfidf.transpose()


def produceCsv():
    # export csv
    dataset = Dataset(load_tags=True,
                      filter_tag=False,
                      weight_tag=False)
    dataset.set_track_attr_weights_2(1.0, 1.0, 0.0, 0.0, 0.0,
                                     1.0, 1.0, 0.0, 0.0)
    cbf_exporter = ContentBasedFiltering()
    urm = dataset.build_train_matrix()
    tg_playlist = list(dataset.target_playlists.keys())
    tg_tracks = list(dataset.target_tracks.keys())
    # Train the model with the best shrinkage found in cross-validation
    cbf_exporter.fit(urm,
                     tg_playlist,
                     tg_tracks,
                     dataset)
    recs = cbf_exporter.predict()
    with open('submission_cbf.csv', mode='w', newline='') as out:
        fieldnames = ['playlist_id', 'track_ids']
        writer = csv.DictWriter(out, fieldnames=fieldnames, delimiter=',')
        writer.writeheader()
        for k in tg_playlist:
            track_ids = ''
            for r in recs[k]:
                track_ids = track_ids + r + ' '
            writer.writerow({'playlist_id': k,
                             'track_ids': track_ids[:-1]})


def evaluateMap():
    from src.utils.evaluator import Evaluator
    dataset = Dataset(load_tags=True,
                      filter_tag=False,
                      weight_tag=False)
    dataset.set_track_attr_weights_2(1.0, 1.0, 0.0, 0.0, 0.0,
                                     1.0, 1.0, 0.0, 0.0)
    # seed = 0xcafebabe
    # print("Evaluating with initial seed: {}".format(seed))
    ev = Evaluator(seed=False)
    ev.cross_validation(5, dataset.train_final.copy())
    cbf = ContentBasedFiltering()
    for i in range(0, 5):
        urm, tg_tracks, tg_playlist = ev.get_fold(dataset)
        cbf.fit(urm,
                list(tg_playlist),
                list(tg_tracks),
                dataset)
        recs = cbf.predict()
        ev.evaluate_fold(recs)

    map_at_five = ev.get_mean_map()
    print("MAP@5 ", map_at_five)


def crossValidation():
    from src.utils.evaluator import Evaluator
    pass

# Item Based Similarity

In [4]:
def ibs(urm, dataset):
    urm_sim = applyTFIDF(urm)
    #urm_sim = top_k_filtering(urm_sim.transpose()).transpose()
    S = compute_cosine(urm_sim.transpose()[[dataset.get_track_index_from_id(x)
                                    for x in tg_tracks]],
                       urm_sim,
                       k_filtering=100,
                       shrinkage=10,
                       n_threads=4,
                       chunksize=1000)
    s_norm = S.sum(axis=1)
    s_norm[s_norm==0] = 1
    # Normalize S
    S = S.multiply(csr_matrix(np.reciprocal(s_norm)))
    print("Similarity matrix ready!")
    return S.transpose()

# User Based Similarity

In [5]:
def ubs(urm, ucm, dataset):
    ucm = dataset.build_ucm()
    iucm = ucm.dot(urm)
    iucm = applyTFIDF(iucm)
    iucm = top_k_filtering(iucm.transpose(), 100).transpose()
    S = compute_cosine(iucm.transpose()[[dataset.get_track_index_from_id(x)
                                                for x in tg_tracks]],
                           iucm,
                           k_filtering=100,
                           shrinkage=10,
                           n_threads=4,
                           chunksize=1000)
    s_norm = S.sum(axis=1)
    s_norm[s_norm==0] = 1
    print("S done")
    # Normalize S
    S = S.multiply(csr_matrix(np.reciprocal(s_norm)))
    print("Similarity matrix ready!")
    return S.transpose()

# Only Content Based

In [6]:
def icm_sim(icm, dataset):
    icm_tag = dataset.build_tags_matrix()
    #tag_weight = np.array(urm.dot(icm_tag.transpose()).sum(axis=0)).squeeze()
    #print(tag_weight[tag_weight>5].shape)
    #icm_tag = icm_tag[tag_weight>5].multiply(tag_weight[tag_weight>5])
    tags = applyTFIDF(icm_tag, norm='l1')
    #tags = icm_tag[tag_weight>5]
    # Before stacking tags with the rest of the ICM, we keep only
    # the top K tags for each item. This way we try to reduce the
    # natural noise added by such sparse features.
    tags = top_k_filtering(tags.transpose(), topK=55).transpose()
    #tags.data = tags.data / np.max(tags.data)
    #newvalue= (max'-min')/(max-min)*(value-max)+max'

    # User augmented UCM
    # print("Building User augmented ICM")
    # ucm = dataset.build_ucm()
    # ua_icm = user_augmented_icm(urm, ucm)
    # ua_icm = top_k_filtering(ua_icm.transpose(), topK=55).transpose()

    # stack all
    icm = vstack([icm, tags*0.4], format='csr')
    #icm = applyTFIDF(icm)
    #icm = top_k_filtering(icm.transpose(), topK=200).transpose()
    # icm = vstack([icm, tags, applyTFIDF(urm)], format='csr')

    S = compute_cosine(icm.transpose()[[dataset.get_track_index_from_id(x)
                                        for x in tg_tracks]],
                   icm,
                   k_filtering=100,
                   shrinkage=10,
                   n_threads=4,
                   chunksize=1000)
    s_norm = S.sum(axis=1)
    s_norm[s_norm==0] = 1
    print("S done")
    # Normalize S
    S = S.multiply(csr_matrix(np.reciprocal(s_norm)))
    print("Similarity matrix ready!")
    return S.transpose()

In [7]:
def icm_sim2(icm, dataset):

    S = compute_cosine(icm.transpose()[[dataset.get_track_index_from_id(x)
                                        for x in tg_tracks]],
                   icm,
                   k_filtering=100,
                   shrinkage=10,
                   n_threads=4,
                   chunksize=1000)
    s_norm = S.sum(axis=1)
    s_norm[s_norm==0] = 1
    print("S done")
    # Normalize S
    S = S.multiply(csr_matrix(np.reciprocal(s_norm)))
    print("Similarity matrix ready!")
    return S.transpose()

# MF-Based Similarity

In [8]:
def mf_sim(urm, dataset, tg_playlist, tg_tracks):
    from src.CBF.CBF_MF import ContentBasedFiltering as CBF
    from src.MF.MF_BPR.MF_BPR import MF_BPR
    cbf = CBF()

    cbf.fit(urm, tg_playlist,
            tg_tracks,
            dataset)

    # get R_hat
    R_hat_aug = cbf.getR_hat()
    print(R_hat_aug.nnz)
    
    mf = MF_BPR()
    
    print("Started learning MF")
    # MAP@5: 0.08256503053607782 with 500 factors after 10 epochs
    # MAP@5: 0.08594586443489391 with 500 factors afetr 4 epochs no_components=500, epoch_multiplier=2, l_rate=1e-2
    mf.fit(R_hat_aug, dataset, list(tg_playlist), list(tg_tracks), n_epochs=1, no_components=500, epoch_multiplier=6, l_rate=1e-2, use_icm=False)
    #recs = mf.predict_dot_custom(urm)
    #ev.evaluate_fold(recs)

    # MAP@5: 0.09407901681369218 with neighborhood
    # MAP@5: 0.  with neighborhood after 4 epochs
    # MAP@5 after 2 epoch with epoch mult = 3 (6 epochs)  0.09326999924602265 with 500 factors,
    # Urm augmented by 5 each user by cbf.
    #recs = mf.predict_knn_custom(urm)
    #ev.evaluate_fold(recs)

    return mf.S

# SLIM Similarity

In [9]:
from src.ML.BPRSLIM import BPRSLIM
import os


dataset.set_track_attr_weights_2(1, 1, 0, 0, 0, 0, 0, 0, 0)
#ds.set_track_attr_weights(1, 1, 0, 0, 1)
print('Building the ICM...')
icm = dataset.build_icm()

icm_tag = dataset.build_tags_matrix()

tags = applyTFIDF(icm_tag, norm='l1')

# Before stacking tags with the rest of the ICM, we keep only
# the top K tags for each item. This way we try to reduce the
# natural noise added by such sparse features.
tags = top_k_filtering(tags.transpose(), topK=55).transpose()
tags.data = np.ones_like(tags.data)

# stack all
icm = vstack([icm, tags], format='csr')

recommender = BPRSLIM(epochs=50,
                      epochMultiplier=1.0,
                      sgd_mode='rmsprop',
                      learning_rate=5e-02,
                      topK=300,
                      urmSamplingChances=1 / 5,
                      icmSamplingChances=4 / 5)
recommender.set_evaluation_every(1, ev)
recommender.fit(urm.tocsr(),
                icm.tocsr(),
                tg_playlist,
                tg_tracks,
                dataset)

S_bprslim = recommender.getParameters()
S_bprslim = S_bprslim[:,[dataset.get_track_index_from_id(x) for x in tg_tracks]]

# keep only 100 elements
S_bprslim = top_k_filtering(S_bprslim.transpose(), 100).transpose()

# normalize
s_norm = S_bprslim.sum(axis=0)
s_norm[s_norm==0] = 1
print("S done")

S_bprslim = S_bprslim.multiply(csr_matrix(np.reciprocal(s_norm)))


Building the ICM...
Running fit process.
Processed 500000 ( 29.85% ) in 23.64 seconds. Sample per second: 21153
Processed 1000000 ( 59.70% ) in 8.64 seconds. Sample per second: 31606
Processed 1500000 ( 89.54% ) in 7.28 seconds. Sample per second: 39189
Processed 1675142 ( 100.00% ) in 2.72 seconds. Sample per second: 41140
Return S matrix to python caller...
Epoch 0 of 50 complete in 0.87 minutes
Processed 500000 ( 29.85% ) in 7.59 seconds. Sample per second: 65876
Processed 1000000 ( 59.70% ) in 6.32 seconds. Sample per second: 75075
Processed 1500000 ( 89.54% ) in 5.85 seconds. Sample per second: 79592
Processed 1675142 ( 100.00% ) in 2.75 seconds. Sample per second: 80739
Return S matrix to python caller...
Evaluating recommendations
R_hat evaluated...


/Users/emanueleghelfi/Development/RS/lib/python3.6/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


MAP@5: 0.06886375631455911
Epoch 1 of 50 complete in 0.90 minutes
Processed 500000 ( 29.85% ) in 6.26 seconds. Sample per second: 79876
Processed 1000000 ( 59.70% ) in 6.28 seconds. Sample per second: 81458
Processed 1500000 ( 89.54% ) in 6.10 seconds. Sample per second: 82880
Processed 1675142 ( 100.00% ) in 2.06 seconds. Sample per second: 83498
Return S matrix to python caller...
Evaluating recommendations
R_hat evaluated...
MAP@5: 0.07091306642539365
Epoch 2 of 50 complete in 0.94 minutes
Processed 500000 ( 29.85% ) in 6.28 seconds. Sample per second: 79597
Processed 1000000 ( 59.70% ) in 5.78 seconds. Sample per second: 84867
Processed 1500000 ( 89.54% ) in 6.35 seconds. Sample per second: 86449
Processed 1675142 ( 100.00% ) in 2.20 seconds. Sample per second: 87261
Return S matrix to python caller...
Evaluating recommendations
R_hat evaluated...
MAP@5: 0.07018246248963242
Epoch 3 of 50 complete in 1.03 minutes
Processed 500000 ( 29.85% ) in 6.12 seconds. Sample per second: 81740


KeyboardInterrupt: 

# Initialization

In [10]:
from src.utils.evaluator import Evaluator
from src.utils.loader import *
print("started")
print("hey")
dataset = Dataset(load_tags=True,
                  filter_tag=False,
                  weight_tag=False)
dataset.set_track_attr_weights_2(1.5, 1.6, 0, 0, 0.0,
                                 1.0, 0, 0.0, 0.0)
# seed = 0xcafebabe
# print("Evaluating with initial seed: {}".format(seed))
ev = Evaluator(seed=False)
ev.cross_validation(5, dataset.train_final.copy())
urm, tg_tracks, tg_playlist = ev.get_fold(dataset)
print("Done")

started
hey
File found, retrieving urm from it.
Load from file takes 0.31 seconds
Done


In [106]:
print("Start exporting")
dataset = Dataset(load_tags=True,
                  filter_tag=False,
                  weight_tag=False)
dataset.set_track_attr_weights_2(1.5, 1.6, 0, 0, 0.0,
                                 1.0, 0, 0.0, 0.0)
# seed = 0xcafebabe
# print("Evaluating with initial seed: {}".format(seed))
urm = dataset.build_train_matrix()
tg_playlist = list(dataset.target_playlists.keys())
tg_tracks = list(dataset.target_tracks.keys())

Start exporting
File found, retrieving urm from it.
Load from file takes 0.68 seconds


# Ensemble

In [33]:
print("ok")
cbf = ContentBasedFiltering()
cbf.fit(urm,
        list(tg_playlist),
        list(tg_tracks),
        dataset, 
        topK_tag=55)
recs = cbf.predict()
ev.evaluate_fold(recs)
S_cbf_full = cbf.S

ok
CBF started
Build tags matrix and apply TFIDF...
Running 4 workers...
[ 4284 ] Building cosine similarity matrix for [0, 1000)...
[ 4285 ] Building cosine similarity matrix for [4298, 5298)...
[ 4286 ] Building cosine similarity matrix for [8596, 9596)...
[ 4287 ] Building cosine similarity matrix for [12894, 13894)...
[ 4284 ] Building cosine similarity matrix for [1000, 2000)...
[ 4285 ] Building cosine similarity matrix for [5298, 6298)...
[ 4286 ] Building cosine similarity matrix for [9596, 10596)...
[ 4287 ] Building cosine similarity matrix for [13894, 14894)...
[ 4284 ] Building cosine similarity matrix for [2000, 3000)...
[ 4285 ] Building cosine similarity matrix for [6298, 7298)...
[ 4286 ] Building cosine similarity matrix for [10596, 11596)...
[ 4287 ] Building cosine similarity matrix for [14894, 15894)...
[ 4284 ] Building cosine similarity matrix for [3000, 4000)...
[ 4285 ] Building cosine similarity matrix for [7298, 8298)...
[ 4286 ] Building cosine similarity mat

In [34]:
S_mf = mf_sim(urm, dataset, tg_playlist, tg_tracks)

CBF started
Build tags matrix and apply TFIDF...
Running 4 workers...
[ 4292 ] Building cosine similarity matrix for [0, 1000)...
[ 4293 ] Building cosine similarity matrix for [25000, 26000)...
[ 4292 ] Building cosine similarity matrix for [1000, 2000)...
[ 4294 ] Building cosine similarity matrix for [50000, 51000)...
[ 4293 ] Building cosine similarity matrix for [26000, 27000)...
[ 4292 ] Building cosine similarity matrix for [2000, 3000)...
[ 4295 ] Building cosine similarity matrix for [75000, 76000)...
[ 4293 ] Building cosine similarity matrix for [27000, 28000)...
[ 4292 ] Building cosine similarity matrix for [3000, 4000)...
[ 4294 ] Building cosine similarity matrix for [51000, 52000)...
[ 4293 ] Building cosine similarity matrix for [28000, 29000)...
[ 4292 ] Building cosine similarity matrix for [4000, 5000)...
[ 4295 ] Building cosine similarity matrix for [76000, 77000)...
[ 4294 ] Building cosine similarity matrix for [52000, 53000)...
[ 4292 ] Building cosine similari

/Users/emanueleghelfi/Development/RS/lib/python3.6/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


Shape of final matrix:  (57561, 100000)
1246608
Started learning MF
Processed 5000000 ( 66.85% ) in 496.54 seconds. Sample per second: 10070
Processed 7479648 ( 100.00% ) in 242.96 seconds. Sample per second: 10122
Training finished
Computing dot product for chunk [0, 1000)...
Computing dot product for chunk [1000, 2000)...
Computing dot product for chunk [2000, 3000)...
Computing dot product for chunk [3000, 4000)...
Computing dot product for chunk [4000, 5000)...
Computing dot product for chunk [5000, 6000)...
Computing dot product for chunk [6000, 7000)...
Computing dot product for chunk [7000, 8000)...
Computing dot product for chunk [8000, 9000)...
Computing dot product for chunk [9000, 10000)...
Computing dot product for chunk [10000, 11000)...
Computing dot product for chunk [11000, 12000)...
Computing dot product for chunk [12000, 13000)...
Computing dot product for chunk [13000, 14000)...
Computing dot product for chunk [14000, 15000)...
Computing dot product for chunk [15000,

In [35]:
dataset.set_playlist_attr_weights(1, 1, 1, 0, 0)
ucm = dataset.build_ucm()
S_user = ubs(urm, ucm, dataset)

Running 4 workers...
[ 4339 ] Building cosine similarity matrix for [0, 1000)...
[ 4340 ] Building cosine similarity matrix for [4298, 5298)...
[ 4341 ] Building cosine similarity matrix for [8596, 9596)...
[ 4342 ] Building cosine similarity matrix for [12894, 13894)...
[ 4339 ] Building cosine similarity matrix for [1000, 2000)...
[ 4340 ] Building cosine similarity matrix for [5298, 6298)...
[ 4341 ] Building cosine similarity matrix for [9596, 10596)...
[ 4342 ] Building cosine similarity matrix for [13894, 14894)...
[ 4339 ] Building cosine similarity matrix for [2000, 3000)...
[ 4340 ] Building cosine similarity matrix for [6298, 7298)...
[ 4341 ] Building cosine similarity matrix for [10596, 11596)...
[ 4342 ] Building cosine similarity matrix for [14894, 15894)...
[ 4339 ] Building cosine similarity matrix for [3000, 4000)...
[ 4340 ] Building cosine similarity matrix for [7298, 8298)...
[ 4341 ] Building cosine similarity matrix for [11596, 12596)...
[ 4342 ] Building cosine s

In [36]:
S_urm = ibs(urm, dataset)

Running 4 workers...
[ 4344 ] Building cosine similarity matrix for [0, 1000)...
[ 4345 ] Building cosine similarity matrix for [4298, 5298)...
[ 4346 ] Building cosine similarity matrix for [8596, 9596)...
[ 4347 ] Building cosine similarity matrix for [12894, 13894)...
[ 4344 ] Building cosine similarity matrix for [1000, 2000)...
[ 4345 ] Building cosine similarity matrix for [5298, 6298)...
[ 4344 ] Building cosine similarity matrix for [2000, 3000)...
[ 4346 ] Building cosine similarity matrix for [9596, 10596)...
[ 4345 ] Building cosine similarity matrix for [6298, 7298)...
[ 4344 ] Building cosine similarity matrix for [3000, 4000)...
[ 4346 ] Building cosine similarity matrix for [10596, 11596)...
[ 4347 ] Building cosine similarity matrix for [13894, 14894)...
[ 4345 ] Building cosine similarity matrix for [7298, 8298)...
[ 4344 ] Building cosine similarity matrix for [4000, 4298)...
[ 4347 ] Building cosine similarity matrix for [14894, 15894)...
[ 4346 ] Building cosine sim

In [37]:
# 1.5, 1.6, 0.1, 0.2, 0.0, 1.0, 0, 0.0, 0.0  MAP 0.102
# 1.5, 1.6, 0.1, 0.2, 0.0, 0.0, 1.0, 0, 0.0 MAP 0.1033928975344946

dataset.set_track_attr_weights_2(1.5, 1.6, 0, 0, 0.0,
                                 0.0, 1.0, 0, 0.0)
S_icm = icm_sim(dataset.build_icm(), dataset)

[ 4348 ] Building cosine similarity matrix for [0, 1000)...
Running 4 workers...
[ 4349 ] Building cosine similarity matrix for [4298, 5298)...
[ 4350 ] Building cosine similarity matrix for [8596, 9596)...
[ 4351 ] Building cosine similarity matrix for [12894, 13894)...
[ 4348 ] Building cosine similarity matrix for [1000, 2000)...
[ 4349 ] Building cosine similarity matrix for [5298, 6298)...
[ 4350 ] Building cosine similarity matrix for [9596, 10596)...
[ 4351 ] Building cosine similarity matrix for [13894, 14894)...
[ 4348 ] Building cosine similarity matrix for [2000, 3000)...
[ 4349 ] Building cosine similarity matrix for [6298, 7298)...
[ 4350 ] Building cosine similarity matrix for [10596, 11596)...
[ 4351 ] Building cosine similarity matrix for [14894, 15894)...
[ 4348 ] Building cosine similarity matrix for [3000, 4000)...
[ 4349 ] Building cosine similarity matrix for [7298, 8298)...
[ 4350 ] Building cosine similarity matrix for [11596, 12596)...
[ 4351 ] Building cosine s

In [38]:
# 0.080286755758088982, 0.18553973793824477, 0.0075749795138103666, 0.011466764550627563, 0.010195035817612831, 0.049304432078911345

dataset.set_track_attr_weights_2(0.080286755758088982, 0.18553973793824477, 0.0075749795138103666, 0.011466764550627563, 0.010195035817612831,
                                 0.0, 0.049304432078911345, 0, 0.0)
S_icm2 = icm_sim2(dataset.build_icm(), dataset)

[ 4354 ] Building cosine similarity matrix for [0, 1000)...
Running 4 workers...
[ 4355 ] Building cosine similarity matrix for [4298, 5298)...
[ 4356 ] Building cosine similarity matrix for [8596, 9596)...
[ 4357 ] Building cosine similarity matrix for [12894, 13894)...
[ 4354 ] Building cosine similarity matrix for [1000, 2000)...
[ 4355 ] Building cosine similarity matrix for [5298, 6298)...
[ 4356 ] Building cosine similarity matrix for [9596, 10596)...
[ 4357 ] Building cosine similarity matrix for [13894, 14894)...
[ 4354 ] Building cosine similarity matrix for [2000, 3000)...
[ 4355 ] Building cosine similarity matrix for [6298, 7298)...
[ 4357 ] Building cosine similarity matrix for [14894, 15894)...
[ 4356 ] Building cosine similarity matrix for [10596, 11596)...
[ 4354 ] Building cosine similarity matrix for [3000, 4000)...
[ 4355 ] Building cosine similarity matrix for [7298, 8298)...
[ 4357 ] Building cosine similarity matrix for [15894, 16894)...
[ 4356 ] Building cosine s

In [19]:
# Best params 0.53119031862654198, 0.8622528471283929, 0.449568357270693, 0.84725181341512967]
# [0.24995989924853496, 0.35681721890423695, 0.12389358733625891, 0.78110404084094665]
# weights = [0.53, 0.86, 0.45, 0.85]
# with MF [0.027625984433804714, 0.028305576119182392, 0.0054533687749983581, 0.17127788497293486, 0.022819397358672337]
# result is 0.12258991178466422 good!
# [0.21601720781243466, 0.16141373641762527, 0.30928573478985127, 0.72121294762063692, 0.037234416396506938]
S = S_icm * 0.21601720781243466 + S_urm * 0.16141373641762527 + S_user * 0.30928573478985127 + S_cbf_full * 0.72121294762063692 + S_mf * 0.037234416396506938 + S_icm2 * 0.02
print("Similarity matrix ready!")

# keep only topk?
S = top_k_filtering(S.transpose(), 300).transpose()
s_norm = S.sum(axis=0)
s_norm[s_norm==0] = 1
# Normalize S
S = S.multiply(csr_matrix(np.reciprocal(s_norm)))

# Keep only the target playlists in the URM
urm_cleaned = urm[[dataset.get_playlist_index_from_id(x)
                   for x in tg_playlist]]

# Compute ratings
R_hat = urm_cleaned.dot(S.tocsc()).tocsr()
print("R_hat done")

# Remove the entries in R_hat that are already present in the URM
urm_cleaned = urm_cleaned[:, [dataset.get_track_index_from_id(x)
                              for x in tg_tracks]]
R_hat[urm_cleaned.nonzero()] = 0
R_hat.eliminate_zeros()

print("Shape of final matrix: ", R_hat.shape)
recs = predict(R_hat, list(tg_playlist), list(tg_tracks))
ev.evaluate_fold(recs)

Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17351)
MAP@5: 0.1033928975344946


0.1033928975344946

In [15]:
def predict(R_hat, pl_id_list, tr_id_list, at=5):
        """
        returns a dictionary of
        'pl_id': ['tr_1', 'tr_at'] for each playlist in target playlist
        """
        recs = {}
        for i in range(0, R_hat.shape[0]):
            pl_id = pl_id_list[i]
            pl_row = R_hat.data[R_hat.indptr[i]:
                                     R_hat.indptr[i + 1]]
            # get top 5 indeces. argsort, flip and get first at-1 items
            sorted_row_idx = np.flip(pl_row.argsort(), axis=0)[0:at]
            track_cols = [R_hat.indices[R_hat.indptr[i] + x]
                          for x in sorted_row_idx]
            tracks_ids = [tr_id_list[x] for x in track_cols]
            recs[pl_id] = tracks_ids
        return recs

In [20]:
recs = predict(R_hat, tg_playlist, tg_tracks)
with open('submission_ensemble.csv', mode='w', newline='') as out:
    fieldnames = ['playlist_id', 'track_ids']
    writer = csv.DictWriter(out, fieldnames=fieldnames, delimiter=',')
    writer.writeheader()
    for k in tg_playlist:
        track_ids = ''
        for r in recs[k]:
            track_ids = track_ids + r + ' '
        writer.writerow({'playlist_id': k,
                         'track_ids': track_ids[:-1]})

# Bayesian opt

In [25]:
def mixS(params):
    
    R_hat = getR_hat_mixed(params)

    recs = predict(R_hat, list(tg_playlist), list(tg_tracks))
    map_at_five = ev.evaluate_fold(recs)
    return -map_at_five

In [ ]:
def getR_hat_mixed(params):
    global S_icm, S_user, S_urm, S_cbf_full, S_mf, urm, tg_playlist, evaluator
    print(params)

    S = S_icm * params[0] + S_urm * params[1] + S_user * params[2] + S_cbf_full * params[3] + S_mf * params[4]
    
    # maybe it's possible to avoid, who knows?
    S = top_k_filtering(S.transpose(), 300).transpose()
    
    s_norm = S.sum(axis=0)
    s_norm[s_norm==0] = 1
    # Normalize S
    S = S.multiply(csr_matrix(np.reciprocal(s_norm)))

    # Keep only the target playlists in the URM
    urm_cleaned = urm[[dataset.get_playlist_index_from_id(x)
                       for x in tg_playlist]]

    # Compute ratings
    R_hat = urm_cleaned.dot(S.tocsc()).tocsr()

    # Remove the entries in R_hat that are already present in the URM
    urm_cleaned = urm_cleaned[:, [dataset.get_track_index_from_id(x)
                                  for x in tg_tracks]]
    R_hat[urm_cleaned.nonzero()] = 0
    R_hat.eliminate_zeros()

    return R_hat

In [26]:
from skopt import forest_minimize
from skopt.space import Integer, Real
from src.utils.evaluator import Evaluator
from src.utils.loader import *

print("started")

# Dataset initialization
dataset = Dataset(load_tags=True,
                  filter_tag=False,
                  weight_tag=False)

# seed = 0xcafebabe
# print("Evaluating with initial seed: {}".format(seed))
cv_size = 5

# number of similarity matrices
S_number = 6
ev = Evaluator(seed=False)
ev.cross_validation(cv_size, dataset.train_final.copy())

# if we want to use SLIM we need to add it
space = [Real(0.0, 1.0),  # ICM
         Real(0.0, 1.0),  # URM
         Real(0.0, 1.0),  # USER
         Real(0.0, 1.0),  # CBF_FULL
         Real(0.0, 1.0),  # MF
         Real(0.0, 1.0),  # ICM2
             ]
x0 = [1, 0, 0, 0, 0, 0]
x1 = [0, 1, 0, 0, 0, 0]
x2 = [0, 0, 1, 0, 0, 0]
x3 = [0, 0, 0, 1, 0, 0]
x4 = [0.53119031862654198, 0.8622528471283929, 0.449568357270693, 0.84725181341512967, 0, 0]
x5 = [0, 0, 0, 0, 1, 0]
x6 = [0.027625984433804714, 0.028305576119182392, 0.0054533687749983581, 0.17127788497293486, 0.022819397358672337, 0]
x0s = [x0, x1, x2, x3, x4, x5, x6]

# matrix of weights for doing averages
# columns are weights of a fold, rows are the same weight for different folds
# do the mean over columns
weights = np.zeros((6,cv_size))

for i in range(5):
    
    urm, tg_tracks, tg_playlist = ev.get_fold(dataset)
    
    # set weights
    dataset.set_track_attr_weights_2(1.5, 1.6, 0, 0, 0.0,
                                 1.0, 0, 0.0, 0.0)
    
    # get the similarity matrices
    S_mf = mf_sim(urm, dataset, tg_playlist, tg_tracks)

    print("cbf")
    cbf = ContentBasedFiltering()
    cbf.fit(urm,
            list(tg_playlist),
            list(tg_tracks),
            dataset, 
            topK_tag=55)
    S_cbf_full = cbf.S

    dataset.set_playlist_attr_weights(1, 1, 1, 0, 0)
    ucm = dataset.build_ucm()
    S_user = ubs(urm, ucm, dataset)

    dataset.set_track_attr_weights_2(0.080286755758088982, 0.18553973793824477, 0.0075749795138103666, 0.011466764550627563, 0.010195035817612831,
                                     0.0, 0.049304432078911345, 0, 0.0)
    S_icm2 = icm_sim2(dataset.build_icm(), dataset)

    dataset.set_track_attr_weights_2(1.5, 1.6, 0, 0, 0.0,
                                     0.0, 1.0, 0, 0.0)
    S_icm = icm_sim(dataset.build_icm(), dataset)

    S_urm = ibs(urm, dataset)
    
    
    # Optimize
    res = forest_minimize(mixS, space, x0=x0s, verbose=True, n_random_starts=50, n_calls=1000, n_jobs=-1)
    print('Maximimum p@k found: {:6.5f}'.format(-res.fun))
    print('Optimal parameters:')
    params = ['ICM', 'URM', 'USER', 'FULL']
    for (p, x_) in zip(params, res.x):
        print('{}: {}'.format(p, x_))
        
    # save into the matrix
    weights[:,i] = np.array(res.x)
    
# CV is finished, do the average over columns
weights_final = np.array(np.mean(weights, axis=1))

# exporting
urm = dataset.build_train_matrix()
tg_playlist = list(dataset.target_playlists.keys())
tg_tracks = list(dataset.target_tracks.keys())

# get the similarity matrices
dataset.set_track_attr_weights_2(1.5, 1.6, 0, 0, 0.0,
                             1.0, 0, 0.0, 0.0)

# get the similarity matrices
S_mf = mf_sim(urm, dataset, tg_playlist, tg_tracks)

print("cbf")
cbf = ContentBasedFiltering()
cbf.fit(urm,
        list(tg_playlist),
        list(tg_tracks),
        dataset, 
        topK_tag=55)
S_cbf_full = cbf.S

dataset.set_playlist_attr_weights(1, 1, 1, 0, 0)
ucm = dataset.build_ucm()
S_user = ubs(urm, ucm, dataset)

dataset.set_track_attr_weights_2(0.080286755758088982, 0.18553973793824477, 0.0075749795138103666, 0.011466764550627563, 0.010195035817612831,
                                 0.0, 0.049304432078911345, 0, 0.0)
S_icm2 = icm_sim2(dataset.build_icm(), dataset)

dataset.set_track_attr_weights_2(1.5, 1.6, 0, 0, 0.0,
                                 0.0, 1.0, 0, 0.0)
S_icm = icm_sim(dataset.build_icm(), dataset)

S_urm = ibs(urm, dataset)

R_hat = getR_hat_mixed(weights_final)
    
recs = predict(R_hat, tg_playlist, tg_tracks)
with open('submission_ensemble.csv', mode='w', newline='') as out:
    fieldnames = ['playlist_id', 'track_ids']
    writer = csv.DictWriter(out, fieldnames=fieldnames, delimiter=',')
    writer.writeheader()
    for k in tg_playlist:
        track_ids = ''
        for r in recs[k]:
            track_ids = track_ids + r + ' '
        writer.writerow({'playlist_id': k,
                         'track_ids': track_ids[:-1]})

Iteration No: 1 started. Evaluating function at provided point.
[1, 0, 0, 0, 0]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.10430596395988852
[0, 1, 0, 0, 0]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.08958531252356168
[0, 0, 1, 0, 0]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.07772751262911835
[0, 0, 0, 1, 0]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.12009500113096588
[0.531190318626542, 0.8622528471283929, 0.449568357270693, 0.8472518134151297, 0]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11996757897911488
[0, 0, 0, 0, 1]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.09148759707456817
[0.027625984433804714, 0.028305576119182392, 0.005453368774998358, 0.17127788497293486, 0.022819397358672337]
Similarity matrix ready!
R_hat done
Shape of final matrix:

Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11881323984015693
Iteration No: 19 ended. Evaluation done at random point.
Time taken: 24.2869
Function value obtained: -0.1188
Current minimum: -0.1218
Iteration No: 20 started. Evaluating function at random point.
[0.099737562670298568, 0.20119381721619517, 0.08649259150128398, 0.12783417535646938, 0.622874316951467]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.1128259066576191
Iteration No: 20 ended. Evaluation done at random point.
Time taken: 24.2844
Function value obtained: -0.1128
Current minimum: -0.1218
Iteration No: 21 started. Evaluating function at random point.
[0.35152993543906991, 0.77717004161757663, 0.1330860317250421, 0.58971369793576267, 0.54841939396927608]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.1181135489708213
Iteration No: 21 ended. Evaluation done at random point.
Time taken: 23.9294
Function value o

Iteration No: 39 ended. Evaluation done at random point.
Time taken: 18.6184
Function value obtained: -0.1178
Current minimum: -0.1218
Iteration No: 40 started. Evaluating function at random point.
[0.87838315520772092, 0.038095952761533443, 0.46319594420160737, 0.73112199132675859, 0.1572122135007771]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11789941943753325
Iteration No: 40 ended. Evaluation done at random point.
Time taken: 18.7813
Function value obtained: -0.1179
Current minimum: -0.1218
Iteration No: 41 started. Evaluating function at random point.
[0.14371346654380038, 0.75758651073453387, 0.90763147087956653, 0.93903179593979402, 0.40036155640140558]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11743044560054293
Iteration No: 41 ended. Evaluation done at random point.
Time taken: 18.6169
Function value obtained: -0.1174
Current minimum: -0.1218
Iteration No: 42 started. Evaluating function at random

Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11354218502601217
Iteration No: 60 ended. Search finished for the next optimal point.
Time taken: 19.4248
Function value obtained: -0.1135
Current minimum: -0.1218
Iteration No: 61 started. Searching for the next optimal point.
[5.3312886342693502e-05, 0.56429539691292352, 0.39967593185521366, 0.65341208350337732, 0.2833146112367822]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11451858553871672
Iteration No: 61 ended. Search finished for the next optimal point.
Time taken: 18.9452
Function value obtained: -0.1145
Current minimum: -0.1218
Iteration No: 62 started. Searching for the next optimal point.
[0.032711861220206757, 0.0022050858942574973, 0.13749147681652887, 0.0087612623373933846, 0.24179303668595747]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.10893538415139871
Iteration No: 62 ended. Search finished for the next optim

Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.12065294428108272
Iteration No: 80 ended. Search finished for the next optimal point.
Time taken: 19.0731
Function value obtained: -0.1207
Current minimum: -0.1218
Iteration No: 81 started. Searching for the next optimal point.
[0.0005381545360371166, 0.10033415239014579, 0.96006445874473423, 0.51809962686256816, 0.02935415581853807]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11087913745004903
Iteration No: 81 ended. Search finished for the next optimal point.
Time taken: 19.0285
Function value obtained: -0.1109
Current minimum: -0.1218
Iteration No: 82 started. Searching for the next optimal point.
[0.052542161653094013, 0.93825887600488467, 0.28667955610020762, 0.17805222286288894, 0.0016272736535966064]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.10518057754655818
Iteration No: 82 ended. Search finished for the next optimal

...

Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.10321345095378123
Iteration No: 900 ended. Search finished for the next optimal point.
Time taken: 18.8061
Function value obtained: -0.1032
Current minimum: -0.1218
Iteration No: 901 started. Searching for the next optimal point.
[0.11960660980230967, 0.6975987352675963, 0.48029711421886656, 0.0084012932081728459, 0.032636482864131573]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.10790997511875144
Iteration No: 901 ended. Search finished for the next optimal point.
Time taken: 18.7404
Function value obtained: -0.1079
Current minimum: -0.1218
Iteration No: 902 started. Searching for the next optimal point.
[0.057679494224529906, 0.09756262275960606, 0.53044167512111029, 0.0018127010546213911, 0.019345796305337285]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.1025265776973535
Iteration No: 902 ended. Search finished for the next op

Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.0952522053833973
Iteration No: 920 ended. Search finished for the next optimal point.
Time taken: 18.9858
Function value obtained: -0.0953
Current minimum: -0.1218
Iteration No: 921 started. Searching for the next optimal point.
[0.98028774732607782, 0.97439558443234631, 0.33187483217350494, 6.7566329193802246e-05, 0.024990506106063355]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11679936665912692
Iteration No: 921 ended. Search finished for the next optimal point.
Time taken: 18.5841
Function value obtained: -0.1168
Current minimum: -0.1218
Iteration No: 922 started. Searching for the next optimal point.
[0.0035657040003121985, 0.0065668102076319474, 0.01708318963345901, 0.14048159812108943, 0.20405922360460643]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11346226343964397
Iteration No: 922 ended. Search finished for the next 

Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11835557566161513
Iteration No: 940 ended. Search finished for the next optimal point.
Time taken: 18.6516
Function value obtained: -0.1184
Current minimum: -0.1218
Iteration No: 941 started. Searching for the next optimal point.
[0.35864231397454788, 0.76467187343379328, 0.83873081759632151, 0.00476512391263162, 0.38422066438700953]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11489632813089048
Iteration No: 941 ended. Search finished for the next optimal point.
Time taken: 18.7923
Function value obtained: -0.1149
Current minimum: -0.1218
Iteration No: 942 started. Searching for the next optimal point.
[0.00041630367153311376, 0.14329470096842226, 0.1647098927714469, 0.030823834141943902, 0.22354022440291493]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.10588026841589378
Iteration No: 942 ended. Search finished for the next opti

Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11649928372163172
Iteration No: 960 ended. Search finished for the next optimal point.
Time taken: 18.8808
Function value obtained: -0.1165
Current minimum: -0.1218
Iteration No: 961 started. Searching for the next optimal point.
[0.0013459192041480608, 0.002763091503909521, 0.92828182012753224, 0.0039252054905165598, 0.077349290214155722]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.08394933273015139
Iteration No: 961 ended. Search finished for the next optimal point.
Time taken: 19.9557
Function value obtained: -0.0839
Current minimum: -0.1218
Iteration No: 962 started. Searching for the next optimal point.
[0.27471820471599834, 0.93389300864645353, 0.0044220127881760005, 0.3191924053180068, 0.33661275802081453]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11575284626404299
Iteration No: 962 ended. Search finished for the next 

Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.1171876649325192
Iteration No: 980 ended. Search finished for the next optimal point.
Time taken: 18.7297
Function value obtained: -0.1172
Current minimum: -0.1218
Iteration No: 981 started. Searching for the next optimal point.
[0.0098343624633246991, 0.30804455853060914, 0.15565216327910195, 0.24919584142206824, 0.028351088727541114]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11328055492724119
Iteration No: 981 ended. Search finished for the next optimal point.
Time taken: 18.6727
Function value obtained: -0.1133
Current minimum: -0.1218
Iteration No: 982 started. Searching for the next optimal point.
[0.081237500678162836, 0.2124629193483572, 0.46379200498088158, 0.21019972914143653, 0.033138848201399458]
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17260)
MAP@5: 0.11527407072306425
Iteration No: 982 ended. Search finished for the next opti

# Ensemble user specific

Here we divide the number of ratings in n intervals and we find the correct mix of similarities for every interval

In [68]:
from skopt import forest_minimize
from skopt.space import Integer, Real

# best params [0.60743780798501301, 0.75766752585999741, 0.74643694551203255, 0.92217949239452435, 0.29563731859077508, 0.14148742619410806, 0.52523547719447961, 0.95025892687791746, 0.39335940969308369, 0.32999205674293214, 0.45124766380750581, 0.83841658583852363, 0.53465266748313023, 0.51635807787404442, 0.037395274863386237, 0.68357308283438822, 0.16711621806405755, 0.53400445992963852, 0.69157109724050636, 0.41306956719296062, 0.30457203528448956, 0.95820247202754694, 0.16658999330326466, 0.59519100134429148]

# number of intervals of number of ratings
intervals = 4
S_number = 6
space = [Real(0.0,1.0) for _ in range(S_number)]

space_all = []
for i in range(intervals):
    space_all = space_all + space
res = forest_minimize(mixSComplex, space_all, verbose=True, n_random_starts=100, n_calls=10000, n_jobs=-1)
print('Maximimum p@k found: {:6.5f}'.format(-res.fun))
print('Optimal parameters:')
params = ['ICM', 'URM', 'USER', 'FULL']
for (p, x_) in zip(params, res.x):
    print('{}: {}'.format(p, x_))

Iteration No: 1 started. Evaluating function at random point.
[0.24885532600321919, 0.33124286435142586, 0.30429957824989057, 0.29204634560478754, 0.70723267116860034, 0.77474538476242993, 0.22347708871068489, 0.12037641994528117, 0.25964027014882635, 0.27654989975400218, 0.97697681380577084, 0.26631333730925294, 0.64101379684944348, 0.02748368423816417, 0.31669562962344272, 0.67616621991375858, 0.78341827280282494, 0.27838965937740473, 0.06352134002875777, 0.48413708163331615, 0.49660366867432504, 0.052863307047735578, 0.44330598626900597, 0.49479643405807749]
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final matrix:  (4421, 17257)
MAP@5: 0.11655130815049423
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 160.6006
Function value obtained: -0.1166
Current minimum: -0.1166
Iteration No: 2 started. Evaluating function at random point.
[0.77080785298819521, 0.67249768949973332, 0.65234183568879356

R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final matrix:  (4421, 17257)
MAP@5: 0.11778783080750997
Iteration No: 10 ended. Evaluation done at random point.
Time taken: 141.9318
Function value obtained: -0.1178
Current minimum: -0.1194
Iteration No: 11 started. Evaluating function at random point.
[0.66974303364508525, 0.99305969076784806, 5.4596252081040588e-05, 0.43030547813500764, 0.45619881870144663, 0.82149309400029802, 0.52399160525608623, 0.26624415348769731, 0.50726528493560219, 0.22618962437947893, 0.94142010755371297, 0.54456340596013308, 0.34172390819949722, 0.057135597668685603, 0.001284657158123981, 0.086545395665156746, 0.64987272638552185, 0.90665649850180419, 0.2624184674397983, 0.037473969936688174, 0.93551409900883742, 0.29617364472306823, 0.77239896572494005, 0.89815413184437276]
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final matrix:  (

(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final matrix:  (4421, 17257)
MAP@5: 0.11798763477343002
Iteration No: 20 ended. Evaluation done at random point.
Time taken: 142.9742
Function value obtained: -0.1180
Current minimum: -0.1200
Iteration No: 21 started. Evaluating function at random point.
[0.01292734080448266, 0.7926282645908711, 0.14616433282390343, 0.88270807282841757, 0.7883247286723597, 0.5014596175358117, 0.47762874550491863, 0.8963223826382144, 0.60145363042852706, 0.37896977076737326, 0.11439617111958347, 0.59677336460779185, 0.29550470675480273, 0.1309347214909036, 0.77038761537294886, 0.096669565931990192, 0.11025500338993435, 0.74983143264401886, 0.61314925242741425, 0.7340805027144145, 0.86408712131082732, 0.52765498997783755, 0.40495422161603101, 0.60806080539638108]
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final matri

(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final matrix:  (4421, 17257)
MAP@5: 0.11571816331146806
Iteration No: 30 ended. Evaluation done at random point.
Time taken: 141.3425
Function value obtained: -0.1157
Current minimum: -0.1200
Iteration No: 31 started. Evaluating function at random point.
[0.10805828676909204, 0.46819785709490591, 0.83721579177278493, 0.01626084571991682, 0.25595516651094213, 0.74851964857439834, 0.70371270911262873, 0.48182513646676606, 0.22473975836303378, 0.25331154825076635, 0.40544090202928518, 0.8091199313319114, 0.3845542336310338, 0.40856620357652207, 0.95366151374381192, 0.2861642292482301, 0.75169927439024142, 0.58037251565583958, 0.53592343385063845, 0.39888074077716307, 0.63441859451108695, 0.61027124508008757, 0.59705804005692131, 0.7075304099973494]
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final matr

(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final matrix:  (4421, 17257)
MAP@5: 0.11979567217070068
Iteration No: 40 ended. Evaluation done at random point.
Time taken: 143.1076
Function value obtained: -0.1198
Current minimum: -0.1200
Iteration No: 41 started. Evaluating function at random point.
[0.1193172336031896, 0.68417267541242055, 0.71588995530586075, 0.58912507818653004, 0.85146794818073912, 0.36294628324962858, 0.55317736302716425, 0.85425525553866533, 0.96276589155876846, 0.12522399765124073, 0.066077211564006602, 0.19372982704288694, 0.44048252291280665, 0.62149992219999062, 0.92329594533979154, 0.88685373280903379, 0.63454578731019007, 0.67961636822738369, 0.78232278367458263, 0.77813096078797095, 0.54825092909653794, 0.93642202057176105, 0.82000952178711906, 0.2589438242881818]
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final m

...

(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final matrix:  (4421, 17257)
MAP@5: 0.1169124632436104
Iteration No: 80 ended. Evaluation done at random point.
Time taken: 143.6481
Function value obtained: -0.1169
Current minimum: -0.1204
Iteration No: 81 started. Evaluating function at random point.
[0.56862741353787338, 0.083680033019733494, 0.0020874622408724179, 0.53785866198705845, 0.044811267054590383, 0.59302038637502785, 0.82209909400437109, 0.3733964302246508, 0.2107712037937402, 0.84296714168009901, 0.60579247140091774, 0.91100228097289171, 0.87295240816286734, 0.8315953238421917, 0.076912309537154791, 0.90974661147990954, 0.033596748666012817, 0.12915517972128401, 0.31479641574009481, 0.9356252966144929, 0.46991910127211178, 0.31580920113435668, 0.23827262791920373, 0.84867503343780859]
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final

(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final matrix:  (4421, 17257)
MAP@5: 0.11915780743421578
Iteration No: 90 ended. Evaluation done at random point.
Time taken: 156.1766
Function value obtained: -0.1192
Current minimum: -0.1204
Iteration No: 91 started. Evaluating function at random point.
[0.48113818820047438, 0.064719419685571392, 0.31586069801410238, 0.60278733820804475, 0.87880487504526306, 0.69957550256590273, 0.85441085320153454, 0.70032286188703896, 0.977009270100341, 0.67063426157614137, 0.1798201455049763, 0.17830568792359971, 0.54244124444311936, 0.68490717145619506, 0.79670382679815843, 0.97085247993282919, 0.018388551834205517, 0.63618940389941592, 0.78205829676135374, 0.67433742915186823, 0.24809118408576386, 0.62308680576542563, 0.77345196089507651, 0.96504405147008987]
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final m

(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final matrix:  (4421, 17257)
MAP@5: 0.1186669682575589
Iteration No: 100 ended. Evaluation done at random point.
Time taken: 146.4505
Function value obtained: -0.1187
Current minimum: -0.1204
Iteration No: 101 started. Searching for the next optimal point.
[0.022805100538983773, 0.74739084510155795, 0.9588051126351349, 0.91729340365757828, 0.15841790052650997, 0.027748635444192378, 0.27076693194508578, 0.21155884096711924, 0.3164429212594369, 0.59684786727727923, 0.63217296458474748, 0.93170594744664181, 0.49491265024929154, 0.49078777350977831, 0.80144049440588616, 0.74982827550037157, 0.2123846156738039, 0.30777259041235444, 0.77523921520706041, 0.31974343547330292, 0.029254583678066999, 0.96745511540113627, 0.39797816843005107, 0.89863702500286646]
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of fina

(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
Shape of final matrix:  (4421, 17257)
MAP@5: 0.11673754052627616
Iteration No: 110 ended. Search finished for the next optimal point.
Time taken: 228.6684
Function value obtained: -0.1167
Current minimum: -0.1204
Iteration No: 111 started. Searching for the next optimal point.
[0.27838098821523954, 0.7975694647469509, 0.9445288899857085, 0.45601537009649307, 0.025361945405315563, 0.0048457505165416812, 0.04336124703855649, 0.5779529717162688, 0.038629196602532063, 0.8950952787499078, 0.032726711913735669, 0.30602111584368014, 0.80974905798221364, 0.30863838124743154, 0.26535860199041611, 0.076545781623942113, 0.0036686493965233389, 0.35828298062947356, 0.97863744152081933, 0.41513177801932188, 0.43711931060311049, 0.044875835297942708, 0.30838412291410527, 0.1042501909961541]
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat done
(100000, 17257)
R_hat do

KeyboardInterrupt: 

In [18]:
def mixSComplex(params, cluster_type='n_ratings'):
    """
    cluster_type can be n_ratings or n_features
    """
    global S_icm, S_user, S_urm, S_cbf_full, S_mf, urm, tg_playlist, evaluator
    print(params)
    # list of matrices
    Ss = [S_icm, S_user, S_urm, S_cbf_full, S_mf, S_icm2]
    
    # this is a list of matrix mixed with theirs params
    Ss_mixed = []
    
    for i in range(intervals):
        # get the correct params
        params_ = params[i*S_number:(i+1)*S_number]
        S_cur = lil_matrix(Ss[0].shape)
        for j in range(len(Ss)):
            S_cur += params_[j] * Ss[j]
        # maybe it's possible to avoid, who knows?
        S_cur = top_k_filtering(S_cur.transpose(), 300).transpose()

        s_norm = S_cur.sum(axis=0)
        s_norm[s_norm==0] = 1
        # Normalize S
        S_cur = S_cur.multiply(csr_matrix(np.reciprocal(s_norm)))
        Ss_mixed.append(S_cur)
    
    # R_hat computations
    # Keep only the target playlists in the URM
    urm_cleaned = urm[[dataset.get_playlist_index_from_id(x)
                       for x in tg_playlist]]
    # Remove the entries in R_hat that are already present in the URM
    urm_cleaned_t = urm_cleaned[:, [dataset.get_track_index_from_id(x)
                                    for x in tg_tracks]]
    R_hats = []
    for i in range(intervals):
        print(Ss_mixed[i].shape)
        # Compute ratings
        R_hat = urm_cleaned.dot(Ss_mixed[i].tocsc()).tocsr()
        print("R_hat done")

        
        R_hat[urm_cleaned_t.nonzero()] = 0
        R_hat.eliminate_zeros()
        R_hats.append(R_hat)
    
    if cluster_type=='n_ratings':
        # divide ratings in intervals
        n_ratings = np.array(urm.sum(axis=1)).squeeze()
        clusters = pd.cut(n_ratings, intervals, labels=False)
        clusters = clusters[[dataset.get_playlist_index_from_id(x)
                           for x in tg_playlist]]
    else:
        dataset.set_track_attr_weights_2(1,1,1,1,0,0,0,0,0)
        icm_full = dataset.build_icm()
        n_feat = np.array(urm.dot(icm_full.transpose()).sum(axis=1)).squeeze()
        clusters = pd.cut(n_feat, intervals, labels=False)
        clusters = clusters[[dataset.get_playlist_index_from_id(x)
                           for x in tg_playlist]]
        
    
    R_hat = lil_matrix(R_hats[i].shape)
    
    for i in range(intervals):
        R_hat[clusters==i] = R_hats[i][clusters==i]
    
    R_hat = csr_matrix(R_hat)
    print("Shape of final matrix: ", R_hat.shape)
    #recs = predict(R_hat, list(tg_playlist), list(tg_tracks))
    #map_at_five = ev.evaluate_fold(recs)
    #return -map_at_five
    return R_hat

In [13]:
def mixSCluster(params):
    global S_icm, S_user, S_urm, S_cbf_full, S_mf, urm, tg_playlist, evaluator, cluster_number, clusters
    print(params)
    S = S_icm * params[0] + S_urm * params[1] + S_user * params[2] + S_cbf_full * params[3] + S_mf * params[4] + S_icm2*params[5]
    
    # maybe it's possible to avoid, who knows?
    S = top_k_filtering(S.transpose(), 300).transpose()
    
    s_norm = S.sum(axis=0)
    s_norm[s_norm==0] = 1
    # Normalize S
    S= S.multiply(csr_matrix(np.reciprocal(s_norm)))
    print("Similarity matrix ready!")

    # Keep only the target playlists in the URM
    urm_cleaned = urm[[dataset.get_playlist_index_from_id(x)
                       for x in tg_playlist]]

    urm_cleaned[clusters != cluster_number] = 0
    urm_cleaned = csr_matrix(urm_cleaned)
    # Compute ratings
    R_hat = urm_cleaned.dot(S.tocsc()).tocsr()
    print("R_hat done")

    # Remove the entries in R_hat that are already present in the URM
    urm_cleaned = urm_cleaned[:, [dataset.get_track_index_from_id(x)
                                  for x in tg_tracks]]
    R_hat[urm_cleaned.nonzero()] = 0
    R_hat.eliminate_zeros()

    recs = predict(R_hat, list(tg_playlist), list(tg_tracks))
    map_at_five = ev.evaluate_fold(recs)
    map_playlists = ev.get_map_playlists()
    maps = np.array([map_playlists[x] for x in list(tg_playlist)])
    maps = maps[clusters==cluster_number]
    return -np.mean(maps)

In [ ]:
from skopt import forest_minimize
from skopt.space import Integer, Real

# best params [0.60743780798501301, 0.75766752585999741, 0.74643694551203255, 0.92217949239452435, 0.29563731859077508, 0.14148742619410806, 0.52523547719447961, 0.95025892687791746, 0.39335940969308369, 0.32999205674293214, 0.45124766380750581, 0.83841658583852363, 0.53465266748313023, 0.51635807787404442, 0.037395274863386237, 0.68357308283438822, 0.16711621806405755, 0.53400445992963852, 0.69157109724050636, 0.41306956719296062, 0.30457203528448956, 0.95820247202754694, 0.16658999330326466, 0.59519100134429148]

# number of intervals of number of ratings
S_number = 6
space = [Real(0.0,1.0) for _ in range(S_number)]
             
x0 = [1, 0, 0, 0, 0, 0]
x1 = [0, 1, 0, 0, 0, 0]
x2 = [0, 0, 1, 0, 0, 0]
x3 = [0, 0, 0, 1, 0, 0]
x4 = [0.53119031862654198, 0.8622528471283929, 0.449568357270693, 0.84725181341512967, 0, 0]
x5 = [0, 0, 0, 0, 1, 0]
x6 = [0.027625984433804714, 0.028305576119182392, 0.0054533687749983581, 0.17127788497293486, 0.022819397358672337, 0]
x0s = [x0, x1, x2, x3, x4, x5, x6]
# divide ratings in intervals
intervals = 10
#n_ratings = np.array(urm.sum(axis=1)).squeeze()
#clusters = pd.cut(n_ratings, intervals, labels=False)
#clusters = clusters[[dataset.get_playlist_index_from_id(x)
#                   for x in tg_playlist]]
dataset.set_track_attr_weights_2(1,1,1,1,0,0,0,0,0)
icm_full = dataset.build_icm()
n_feat = np.array(urm.dot(icm_full.transpose()).sum(axis=1)).squeeze()
clusters = pd.cut(n_feat, intervals, labels=False)
clusters = clusters[[dataset.get_playlist_index_from_id(x)
                   for x in tg_playlist]]

for i in range(intervals):
    cluster_number = i
    
    res = forest_minimize(mixSCluster, space, x0=x0s, verbose=True, n_random_starts=30, n_calls=300, n_jobs=-1)
    print('Maximimum p@k found: {:6.5f}'.format(-res.fun))
    print('Optimal parameters for cluster :', i)
    params = ['ICM', 'URM', 'USER', 'FULL', 'MF', 'ICM2']
    for (p, x_) in zip(params, res.x):
        print('{}: {}'.format(p, x_))

Iteration No: 1 started. Evaluating function at provided point.
[1, 0, 0, 0, 0, 0]
Similarity matrix ready!
R_hat done
MAP@5: 0.06385583955364532
[0, 1, 0, 0, 0, 0]
Similarity matrix ready!
R_hat done
MAP@5: 0.05416798612681881
[0, 0, 1, 0, 0, 0]
Similarity matrix ready!
R_hat done
MAP@5: 0.047030083691472394
[0, 0, 0, 1, 0, 0]
Similarity matrix ready!
R_hat done
MAP@5: 0.0711709266380154
[0.531190318626542, 0.8622528471283929, 0.449568357270693, 0.8472518134151297, 0, 0]
Similarity matrix ready!
R_hat done
MAP@5: 0.07051195053909363
[0, 0, 0, 0, 1, 0]
Similarity matrix ready!
R_hat done
MAP@5: 0.0605111965618637
[0.027625984433804714, 0.028305576119182392, 0.005453368774998358, 0.17127788497293486, 0.022819397358672337, 0]
Similarity matrix ready!
R_hat done
MAP@5: 0.07191284023222488
Iteration No: 1 ended. Evaluation done at provided point.
Time taken: 191.2808
Function value obtained: -0.1498
Current minimum: -0.1498
Iteration No: 2 started. Evaluating function at provided point.
[0

Similarity matrix ready!
R_hat done
MAP@5: 0.07051421247078331
Iteration No: 21 ended. Evaluation done at random point.
Time taken: 27.8354
Function value obtained: -0.1468
Current minimum: -0.1498
Iteration No: 22 started. Evaluating function at random point.
[0.66676322759985118, 0.7598975420137023, 0.049734738258824809, 0.034243403770014454, 0.13820683836247774, 0.06101389864277619]
Similarity matrix ready!
R_hat done
MAP@5: 0.06914197391238781
Iteration No: 22 ended. Evaluation done at random point.
Time taken: 27.6914
Function value obtained: -0.1440
Current minimum: -0.1498
Iteration No: 23 started. Evaluating function at random point.
[0.2777591904569861, 0.52199463216341646, 0.36444626348359127, 0.35162761280538563, 0.74930593546591362, 0.0050475483594485402]
Similarity matrix ready!
R_hat done
MAP@5: 0.07084973233808331
Iteration No: 23 ended. Evaluation done at random point.
Time taken: 27.6843
Function value obtained: -0.1475
Current minimum: -0.1498
Iteration No: 24 started

Similarity matrix ready!
R_hat done
MAP@5: 0.067670210359647
Iteration No: 42 ended. Search finished for the next optimal point.
Time taken: 27.8200
Function value obtained: -0.1409
Current minimum: -0.1498
Iteration No: 43 started. Searching for the next optimal point.
[0.5110536020804054, 0.5054466816214519, 0.96843406151009881, 0.0080959890208845992, 0.99802973004285422, 0.0019117266908600388]
Similarity matrix ready!
R_hat done
MAP@5: 0.07000829374952869
Iteration No: 43 ended. Search finished for the next optimal point.
Time taken: 27.8037
Function value obtained: -0.1458
Current minimum: -0.1498
Iteration No: 44 started. Searching for the next optimal point.
[0.01839490899264851, 0.10068334230108258, 0.95256984035626391, 0.58482559226402631, 0.0077827812340316455, 0.040284696021651085]
Similarity matrix ready!
R_hat done
MAP@5: 0.06885621654226032
Iteration No: 44 ended. Search finished for the next optimal point.
Time taken: 28.0601
Function value obtained: -0.1434
Current minim

Similarity matrix ready!
R_hat done
MAP@5: 0.06814898590062564
Iteration No: 63 ended. Search finished for the next optimal point.
Time taken: 28.2523
Function value obtained: -0.1419
Current minimum: -0.1498
Iteration No: 64 started. Searching for the next optimal point.
[0.0050459691152747741, 0.15357407643759372, 0.49055902646183258, 0.056768340705185896, 0.63058286630470117, 0.001898708367631508]
Similarity matrix ready!
R_hat done
MAP@5: 0.06585086330392811
Iteration No: 64 ended. Search finished for the next optimal point.
Time taken: 27.7038
Function value obtained: -0.1371
Current minimum: -0.1498
Iteration No: 65 started. Searching for the next optimal point.
[0.003209520074637485, 0.0075500566076375248, 0.26247863150403467, 0.08507247032817479, 0.094079640954530724, 0.59738581717822958]
Similarity matrix ready!
R_hat done
MAP@5: 0.06658448314860882
Iteration No: 65 ended. Search finished for the next optimal point.
Time taken: 27.8497
Function value obtained: -0.1387
Current 

Similarity matrix ready!
R_hat done
MAP@5: 0.06843474327075315
Iteration No: 84 ended. Search finished for the next optimal point.
Time taken: 27.8468
Function value obtained: -0.1425
Current minimum: -0.1498
Iteration No: 85 started. Searching for the next optimal point.
[0.0010039857059854775, 0.20172277266535804, 0.40741747791785832, 0.031335142875903006, 0.10380308996613832, 0.093470645551142831]
Similarity matrix ready!
R_hat done
MAP@5: 0.06787227625725693
Iteration No: 85 ended. Search finished for the next optimal point.
Time taken: 27.7963
Function value obtained: -0.1413
Current minimum: -0.1498
Iteration No: 86 started. Searching for the next optimal point.
[0.065374673281240817, 0.39112861758103512, 0.98876129318874462, 0.29293881780158221, 0.72146602309588093, 0.020477527134425081]
Similarity matrix ready!
R_hat done
MAP@5: 0.0679959285229585
Iteration No: 86 ended. Search finished for the next optimal point.
Time taken: 27.9058
Function value obtained: -0.1416
Current min

...

Iteration No: 154 ended. Search finished for the next optimal point.
Time taken: 40.4900
Function value obtained: -0.0665
Current minimum: -0.0766
Iteration No: 155 started. Searching for the next optimal point.
[0.89880140015151577, 0.0049562682859288784, 0.15984509741463918, 0.023055528719553967, 0.99693286176273099, 0.50542174230500969]
Similarity matrix ready!
R_hat done
MAP@5: 0.002011611249340269
Iteration No: 155 ended. Search finished for the next optimal point.
Time taken: 39.7132
Function value obtained: -0.0613
Current minimum: -0.0766
Iteration No: 156 started. Searching for the next optimal point.
[0.21647592393236131, 0.22618000848745695, 0.069967811978388653, 0.76427460826382154, 0.22025804297884849, 0.00095436718609365223]
Similarity matrix ready!
R_hat done
MAP@5: 0.002389353841513985
Iteration No: 156 ended. Search finished for the next optimal point.
Time taken: 40.0363
Function value obtained: -0.0729
Current minimum: -0.0766
Iteration No: 157 started. Searching for

Similarity matrix ready!
R_hat done
MAP@5: 0.0021465731734901603
Iteration No: 175 ended. Search finished for the next optimal point.
Time taken: 39.8806
Function value obtained: -0.0654
Current minimum: -0.0766
Iteration No: 176 started. Searching for the next optimal point.
[0.74415365832807789, 0.0028580546520275978, 0.97193733292111351, 0.16291794919446315, 0.040248276376433585, 0.050921213978943128]
Similarity matrix ready!
R_hat done
MAP@5: 0.00243308452084747
Iteration No: 176 ended. Search finished for the next optimal point.
Time taken: 40.2450
Function value obtained: -0.0742
Current minimum: -0.0766
Iteration No: 177 started. Searching for the next optimal point.
[0.019153863568537325, 0.070945507096256591, 0.31588025695513328, 0.00047284352054588485, 0.012443274980862399, 0.031103089012842318]
Similarity matrix ready!
R_hat done
MAP@5: 0.002090024881248586
Iteration No: 177 ended. Search finished for the next optimal point.
Time taken: 40.0425
Function value obtained: -0.06

Iteration No: 195 ended. Search finished for the next optimal point.
Time taken: 39.7766
Function value obtained: -0.0695
Current minimum: -0.0766
Iteration No: 196 started. Searching for the next optimal point.
[0.026467901370437648, 0.05665182320040997, 0.40923141794021778, 0.014094780748646566, 0.32253255308925238, 0.2594360728217367]
Similarity matrix ready!
R_hat done
MAP@5: 0.0019972856819724036
Iteration No: 196 ended. Search finished for the next optimal point.
Time taken: 40.0690
Function value obtained: -0.0609
Current minimum: -0.0766
Iteration No: 197 started. Searching for the next optimal point.
[0.028511663865150479, 0.087098240204646862, 0.44568394124077459, 0.095199406121932367, 0.011379302631432388, 0.043945911434403191]
Similarity matrix ready!
R_hat done
MAP@5: 0.0021352635150418456
Iteration No: 197 ended. Search finished for the next optimal point.
Time taken: 39.9918
Function value obtained: -0.0651
Current minimum: -0.0766
Iteration No: 198 started. Searching fo

Similarity matrix ready!
R_hat done
MAP@5: 0.0018502601221443108
Iteration No: 216 ended. Search finished for the next optimal point.
Time taken: 40.0358
Function value obtained: -0.0564
Current minimum: -0.0766
Iteration No: 217 started. Searching for the next optimal point.
[0.42927693198618966, 0.40206324516637382, 0.41443627139182515, 0.53719624358484674, 0.99972522921417395, 0.51533823400670753]
Similarity matrix ready!
R_hat done
MAP@5: 0.001963356706627459
Iteration No: 217 ended. Search finished for the next optimal point.
Time taken: 39.8237
Function value obtained: -0.0599
Current minimum: -0.0766
Iteration No: 218 started. Searching for the next optimal point.
[0.0021829440423468687, 0.77451587750436202, 0.17578869975581826, 0.0084978926160130825, 0.091364874768066554, 0.19802528980306594]
Similarity matrix ready!
R_hat done
MAP@5: 0.0020636356782025183
Iteration No: 218 ended. Search finished for the next optimal point.
Time taken: 40.2002
Function value obtained: -0.0629
C

Iteration No: 236 ended. Search finished for the next optimal point.
Time taken: 42.8749
Function value obtained: -0.0636
Current minimum: -0.0766
Iteration No: 237 started. Searching for the next optimal point.
[0.1524365351602526, 0.41060429633096474, 0.45591547795160148, 0.49899462898981894, 0.38278779069890745, 0.020632667390046659]
Similarity matrix ready!
R_hat done
MAP@5: 0.0020937947673980244
Iteration No: 237 ended. Search finished for the next optimal point.
Time taken: 44.1175
Function value obtained: -0.0638
Current minimum: -0.0766
Iteration No: 238 started. Searching for the next optimal point.
[0.45370201141326483, 0.019374994090509582, 0.46603517436928232, 0.1605651122328238, 0.98313715747592378, 0.017073328420676884]
Similarity matrix ready!
R_hat done
MAP@5: 0.0019716504561562227
Iteration No: 238 ended. Search finished for the next optimal point.
Time taken: 44.2468
Function value obtained: -0.0601
Current minimum: -0.0766
Iteration No: 239 started. Searching for the

## Cluster rating based

Optimal parameters for cluster : 0
Maximimum p@k found: 0.12219
ICM: 0.009642311670812533
URM: 0.03549722742589868
USER: 0.045874651180874866
FULL: 0.13569872442995415
MF: 0.025973785644864862
ICM2: 0.01860721460454229

Optimal parameters for cluster : 1
Maximimum p@k found: 0.09962
ICM: 0.01845795814184859
URM: 0.11437368099762504
USER: 0.13630459070062287
FULL: 0.23946298281940864
MF: 0.006252094519587638
ICM2: 0.1630197073121197

Optimal parameters for cluster : 2
ICM: 0.04910953440315703
URM: 0.02102449053449829
USER: 0.27662939310362206
FULL: 0.03891830657588625
MF: 0.00692272654014703
ICM2: 0.16538320356003058

Optimal parameters for cluster : 3
ICM: 0.991854064496276
URM: 0.19773235446621368
USER: 0.5340193649465407
FULL: 0.6122101329799564
MF: 0.33315815956824674
ICM2: 0.3214350855406228

## Cluster feature based

ICM: 0.12697947401554857
URM: 2.2624817918148746e-05
USER: 0.2612768166834523
FULL: 0.3972451242644902
MF: 0.005815582223616667
ICM2: 0.044005905357967674

Optimal parameters for cluster : 1
ICM: 0.4952705927727632
URM: 0.46893064182039423
USER: 0.9378911573590663
FULL: 0.682282692791146
MF: 0.2278406131200394
ICM2: 0.17688966361329908

Optimal parameters for cluster : 2
ICM: 0.941685948773891
URM: 0.09894009274639018
USER: 0.5301183211113225
FULL: 0.14888259713852828
MF: 0.13303899989925475
ICM2: 0.5512671115369258

Optimal parameters for cluster : 3
ICM: 0.04499909553831472
URM: 0.5955876901760268
USER: 0.006337277454504077
FULL: 0.0280267901689294
MF: 0.10434180139819141
ICM2: 0.24200938462059246

## Cluster feature based with 6 clusters

0.4169959196312609
0.6618546778634947
0.8436852569144283
0.5921422837974945
0.2797462955961147
0.07431131353889854


0.5963421426337502
0.3760203975202763
0.7275840128459379
0.4086234694720706
0.31869923855679
0.12632820069622377

0.19359966105061094
0.05280300354472746
0.9701605639338131
0.932950585539636
0.05784694987949203
0.21907059390936207

0.04465656146253672
0.0476209186589739
0.03066419288709766
0.900915738343916
0.002455613689048009
0.049444739088702186

0.00021369901018919451
0.03691450582846468
0.5301125971086408
0.08798096948483115
0.0987976990557812
0.9236459385179002

0.9360331484564114
0.00473618966529188
0.08943679854318044
0.8143330586862819
0.8417627011684025
0.2984094391751602

In [15]:
# export
print("Start exporting")
dataset = Dataset(load_tags=True,
                  filter_tag=False,
                  weight_tag=False)
dataset.set_track_attr_weights_2(1.5, 1.6, 0, 0, 0.0,
                                 1.0, 0, 0.0, 0.0)
# seed = 0xcafebabe
# print("Evaluating with initial seed: {}".format(seed))
urm = dataset.build_train_matrix()
tg_playlist = list(dataset.target_playlists.keys())
tg_tracks = list(dataset.target_tracks.keys())

Start exporting
File found, retrieving urm from it.
Load from file takes 0.41 seconds


In [11]:
S_mf = mf_sim(urm, dataset, tg_playlist, tg_tracks)

print("cbf")
cbf = ContentBasedFiltering()
cbf.fit(urm,
        list(tg_playlist),
        list(tg_tracks),
        dataset, 
        topK_tag=55)
S_cbf_full = cbf.S

dataset.set_playlist_attr_weights(1, 1, 1, 0, 0)
ucm = dataset.build_ucm()
S_user = ubs(urm, ucm, dataset)

dataset.set_track_attr_weights_2(0.080286755758088982, 0.18553973793824477, 0.0075749795138103666, 0.011466764550627563, 0.010195035817612831,
                                 0.0, 0.049304432078911345, 0, 0.0)
S_icm2 = icm_sim2(dataset.build_icm(), dataset)

dataset.set_track_attr_weights_2(1.5, 1.6, 0, 0, 0.0,
                                 0.0, 1.0, 0, 0.0)
S_icm = icm_sim(dataset.build_icm(), dataset)

S_urm = ibs(urm, dataset)

CBF started
Build tags matrix and apply TFIDF...
Running 4 workers...
[ 648 ] Building cosine similarity matrix for [0, 1000)...
[ 649 ] Building cosine similarity matrix for [25000, 26000)...
[ 650 ] Building cosine similarity matrix for [50000, 51000)...
[ 648 ] Building cosine similarity matrix for [1000, 2000)...
[ 651 ] Building cosine similarity matrix for [75000, 76000)...
[ 649 ] Building cosine similarity matrix for [26000, 27000)...
[ 648 ] Building cosine similarity matrix for [2000, 3000)...
[ 650 ] Building cosine similarity matrix for [51000, 52000)...
[ 649 ] Building cosine similarity matrix for [27000, 28000)...
[ 651 ] Building cosine similarity matrix for [76000, 77000)...
[ 648 ] Building cosine similarity matrix for [3000, 4000)...
[ 650 ] Building cosine similarity matrix for [52000, 53000)...
[ 649 ] Building cosine similarity matrix for [28000, 29000)...
[ 648 ] Building cosine similarity matrix for [4000, 5000)...
[ 651 ] Building cosine similarity matrix for [

/Users/emanueleghelfi/Development/RS/lib/python3.6/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


Shape of final matrix:  (57561, 100000)
1246609
Started learning MF
Processed 5000000 ( 66.85% ) in 487.60 seconds. Sample per second: 10254
Processed 7479654 ( 100.00% ) in 242.75 seconds. Sample per second: 10250
Training finished
Computing dot product for chunk [0, 1000)...
Computing dot product for chunk [1000, 2000)...
Computing dot product for chunk [2000, 3000)...
Computing dot product for chunk [3000, 4000)...
Computing dot product for chunk [4000, 5000)...
Computing dot product for chunk [5000, 6000)...
Computing dot product for chunk [6000, 7000)...
Computing dot product for chunk [7000, 8000)...
Computing dot product for chunk [8000, 9000)...
Computing dot product for chunk [9000, 10000)...
Computing dot product for chunk [10000, 11000)...
Computing dot product for chunk [11000, 12000)...
Computing dot product for chunk [12000, 13000)...
Computing dot product for chunk [13000, 14000)...
Computing dot product for chunk [14000, 15000)...
Computing dot product for chunk [15000,

In [19]:
#params = [0.12697947401554857, 2.2624817918148746e-05, 0.2612768166834523, 0.3972451242644902, 0.005815582223616667, 0.044005905357967674, 0.4952705927727632, 0.46893064182039423, 0.9378911573590663, 0.682282692791146, 0.2278406131200394, 0.17688966361329908, 0.941685948773891, 0.09894009274639018, 0.5301183211113225, 0.14888259713852828, 0.13303899989925475, 0.5512671115369258, 0.04499909553831472, 0.5955876901760268, 0.006337277454504077, 0.0280267901689294, 0.10434180139819141, 0.24200938462059246]
params = [0.4169959196312609, 0.6618546778634947, 0.8436852569144283, 0.5921422837974945, 0.2797462955961147, 0.07431131353889854,
0.5963421426337502, 0.3760203975202763, 0.7275840128459379, 0.4086234694720706, 0.31869923855679, 0.12632820069622377,
0.19359966105061094, 0.05280300354472746, 0.9701605639338131, 0.932950585539636, 0.05784694987949203, 0.21907059390936207,
0.04465656146253672, 0.0476209186589739, 0.03066419288709766, 0.900915738343916, 0.002455613689048009, 0.049444739088702186,
0.00021369901018919451, 0.03691450582846468, 0.5301125971086408, 0.08798096948483115, 0.0987976990557812, 0.9236459385179002,
0.9360331484564114, 0.00473618966529188, 0.08943679854318044, 0.8143330586862819, 0.8417627011684025, 0.298409439175160]

intervals = 6

R_hat = mixSComplex(params, cluster_type="n_feat")



[0.4169959196312609, 0.6618546778634947, 0.8436852569144283, 0.5921422837974945, 0.2797462955961147, 0.07431131353889854, 0.5963421426337502, 0.3760203975202763, 0.7275840128459379, 0.4086234694720706, 0.31869923855679, 0.12632820069622377, 0.19359966105061094, 0.05280300354472746, 0.9701605639338131, 0.932950585539636, 0.05784694987949203, 0.21907059390936207, 0.04465656146253672, 0.0476209186589739, 0.03066419288709766, 0.900915738343916, 0.002455613689048009, 0.049444739088702186, 0.00021369901018919451, 0.03691450582846468, 0.5301125971086408, 0.08798096948483115, 0.0987976990557812, 0.9236459385179002, 0.9360331484564114, 0.00473618966529188, 0.08943679854318044, 0.8143330586862819, 0.8417627011684025, 0.29840943917516]
(100000, 32195)
R_hat done
(100000, 32195)
R_hat done
(100000, 32195)
R_hat done
(100000, 32195)
R_hat done
(100000, 32195)
R_hat done
(100000, 32195)
R_hat done
Shape of final matrix:  (10000, 32195)


0.12697947401554857
2.2624817918148746e-05
0.2612768166834523
0.3972451242644902
0.005815582223616667
0.044005905357967674
0.4952705927727632
0.46893064182039423
0.9378911573590663
0.682282692791146
0.2278406131200394
0.17688966361329908
0.941685948773891
0.09894009274639018
0.5301183211113225
0.14888259713852828
0.13303899989925475
0.5512671115369258
0.04499909553831472
0.5955876901760268
0.006337277454504077
0.0280267901689294
0.10434180139819141
0.24200938462059246

# ICM weight Optimization

In [27]:
# BEST PARAMS 0.080286755758088982, 0.18553973793824477, 0.0075749795138103666, 0.011466764550627563, 0.010195035817612831, 0.049304432078911345
def optIcm(params):
    global dataset, ev, tg_playlist, tg_tracks
    print(params)
    artist_weight, album_weight, duration_weight, playcount_weight, tag_weight, inferred_album_weight = params 
    dataset.set_track_attr_weights_2(artist_weight, album_weight, duration_weight, playcount_weight, tag_weight,
                                 0.0, inferred_album_weight, 0, 0.0)
    S = icm_sim2(dataset.build_icm(), dataset)
    # Keep only the target playlists in the URM
    urm_cleaned = urm[[dataset.get_playlist_index_from_id(x)
                       for x in tg_playlist]]

    # Compute ratings
    R_hat = urm_cleaned.dot(S.tocsc()).tocsr()
    print("R_hat done")

    # Remove the entries in R_hat that are already present in the URM
    urm_cleaned = urm_cleaned[:, [dataset.get_track_index_from_id(x)
                                  for x in tg_tracks]]
    R_hat[urm_cleaned.nonzero()] = 0
    R_hat.eliminate_zeros()

    print("Shape of final matrix: ", R_hat.shape)
    recs = predict(R_hat, list(tg_playlist), list(tg_tracks))
    map_at_five = ev.evaluate_fold(recs)
    return -map_at_five

In [28]:
from skopt import forest_minimize
from skopt.space import Integer, Real
space = [Real(0.0, 1.0),  # Artist
         Real(0.0, 1.0),  # Album
         Real(0.0, 1.0),  # Duration
         Real(0.0, 1.0),  # Playcount
         Real(0.0, 1.0),  # Tag
         Real(0.0, 1.0),  # Inf album
             ]
# 1.5, 1.6, 0.1, 0.2, 0.0, 0.0, 1.0, 0, 0.0 
x0 = [0.15, 0.16, 0.01, 0.02, 0, 1]
x0s = [x0]
res = forest_minimize(optIcm, space, x0=x0s, verbose=True, n_random_starts=100, n_calls=10000, n_jobs=-1)
print('Maximimum p@k found: {:6.5f}'.format(-res.fun))
print('Optimal parameters:')
params = ['Artist', 'Album', 'Duration', 'Playcount', 'Tag', 'Inf Album']
for (p, x_) in zip(params, res.x):
    print('{}: {}'.format(p, x_))

Iteration No: 1 started. Evaluating function at provided point.
[0.15, 0.16, 0.01, 0.02, 0, 1]
[ 1374 ] Building cosine similarity matrix for [0, 1000)...
[ 1375 ] Building cosine similarity matrix for [4337, 5337)...
[ 1376 ] Building cosine similarity matrix for [8674, 9674)...
Running 4 workers...
[ 1377 ] Building cosine similarity matrix for [13011, 14011)...
[ 1374 ] Building cosine similarity matrix for [1000, 2000)...
[ 1375 ] Building cosine similarity matrix for [5337, 6337)...
[ 1376 ] Building cosine similarity matrix for [9674, 10674)...
[ 1377 ] Building cosine similarity matrix for [14011, 15011)...
[ 1374 ] Building cosine similarity matrix for [2000, 3000)...
[ 1375 ] Building cosine similarity matrix for [6337, 7337)...
[ 1376 ] Building cosine similarity matrix for [10674, 11674)...
[ 1377 ] Building cosine similarity matrix for [15011, 16011)...
[ 1374 ] Building cosine similarity matrix for [3000, 4000)...
[ 1375 ] Building cosine similarity matrix for [7337, 8337)

[ 1411 ] Building cosine similarity matrix for [16011, 17011)...
[ 1408 ] Building cosine similarity matrix for [4000, 4337)...
[ 1409 ] Building cosine similarity matrix for [8337, 8674)...
[ 1410 ] Building cosine similarity matrix for [12674, 13011)...
[ 1411 ] Building cosine similarity matrix for [17011, 17348)...
[ 1408 ] Building cosine similarity matrix for [17348, 17351)...
S done
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17351)
MAP@5: 0.06339666742064357
Iteration No: 5 ended. Evaluation done at random point.
Time taken: 80.3238
Function value obtained: -0.0634
Current minimum: -0.1025
Iteration No: 6 started. Evaluating function at random point.
[0.13217048676969492, 0.28656451869012939, 0.43178716185897215, 0.6421662438328527, 0.15099363933390778, 0.51653135925230975]
[ 1427 ] Building cosine similarity matrix for [0, 1000)...
Running 4 workers...
[ 1428 ] Building cosine similarity matrix for [4337, 5337)...
[ 1429 ] Building cosine similarity matr

[ 1466 ] Building cosine similarity matrix for [1000, 2000)...
[ 1467 ] Building cosine similarity matrix for [5337, 6337)...
[ 1468 ] Building cosine similarity matrix for [9674, 10674)...
[ 1469 ] Building cosine similarity matrix for [14011, 15011)...
[ 1466 ] Building cosine similarity matrix for [2000, 3000)...
[ 1467 ] Building cosine similarity matrix for [6337, 7337)...
[ 1468 ] Building cosine similarity matrix for [10674, 11674)...
[ 1469 ] Building cosine similarity matrix for [15011, 16011)...
[ 1466 ] Building cosine similarity matrix for [3000, 4000)...
[ 1467 ] Building cosine similarity matrix for [7337, 8337)...
[ 1468 ] Building cosine similarity matrix for [11674, 12674)...
[ 1469 ] Building cosine similarity matrix for [16011, 17011)...
[ 1466 ] Building cosine similarity matrix for [4000, 4337)...
[ 1467 ] Building cosine similarity matrix for [8337, 8674)...
[ 1468 ] Building cosine similarity matrix for [12674, 13011)...
[ 1469 ] Building cosine similarity matrix

[ 1511 ] Building cosine similarity matrix for [17348, 17351)...
S done
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17351)
MAP@5: 0.07534268265098379
Iteration No: 14 ended. Evaluation done at random point.
Time taken: 80.8355
Function value obtained: -0.0753
Current minimum: -0.1040
Iteration No: 15 started. Evaluating function at random point.
[0.47302284255060445, 0.74046748374952454, 0.50901895741738212, 0.05615774034736, 0.99670240279963873, 0.017405427274526434]
[ 1516 ] Building cosine similarity matrix for [0, 1000)...
Running 4 workers...
[ 1517 ] Building cosine similarity matrix for [4337, 5337)...
[ 1518 ] Building cosine similarity matrix for [8674, 9674)...
[ 1519 ] Building cosine similarity matrix for [13011, 14011)...
[ 1516 ] Building cosine similarity matrix for [1000, 2000)...
[ 1517 ] Building cosine similarity matrix for [5337, 6337)...
[ 1518 ] Building cosine similarity matrix for [9674, 10674)...
[ 1519 ] Building cosine similarity matrix

[ 1549 ] Building cosine similarity matrix for [2000, 3000)...
[ 1550 ] Building cosine similarity matrix for [6337, 7337)...
[ 1551 ] Building cosine similarity matrix for [10674, 11674)...
[ 1552 ] Building cosine similarity matrix for [15011, 16011)...
[ 1549 ] Building cosine similarity matrix for [3000, 4000)...
[ 1550 ] Building cosine similarity matrix for [7337, 8337)...
[ 1551 ] Building cosine similarity matrix for [11674, 12674)...
[ 1552 ] Building cosine similarity matrix for [16011, 17011)...
[ 1549 ] Building cosine similarity matrix for [4000, 4337)...
[ 1550 ] Building cosine similarity matrix for [8337, 8674)...
[ 1551 ] Building cosine similarity matrix for [12674, 13011)...
[ 1552 ] Building cosine similarity matrix for [17011, 17348)...
[ 1549 ] Building cosine similarity matrix for [17348, 17351)...
S done
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17351)
MAP@5: 0.08850109326698341
Iteration No: 19 ended. Evaluation done at random point.
Ti

...

[ 9743 ] Building cosine similarity matrix for [5337, 6337)...
[ 9744 ] Building cosine similarity matrix for [9674, 10674)...
[ 9745 ] Building cosine similarity matrix for [14011, 15011)...
[ 9742 ] Building cosine similarity matrix for [2000, 3000)...
[ 9743 ] Building cosine similarity matrix for [6337, 7337)...
[ 9744 ] Building cosine similarity matrix for [10674, 11674)...
[ 9745 ] Building cosine similarity matrix for [15011, 16011)...
[ 9743 ] Building cosine similarity matrix for [7337, 8337)...
[ 9742 ] Building cosine similarity matrix for [3000, 4000)...
[ 9744 ] Building cosine similarity matrix for [11674, 12674)...
[ 9745 ] Building cosine similarity matrix for [16011, 17011)...
[ 9743 ] Building cosine similarity matrix for [8337, 8674)...
[ 9744 ] Building cosine similarity matrix for [12674, 13011)...
[ 9742 ] Building cosine similarity matrix for [4000, 4337)...
[ 9745 ] Building cosine similarity matrix for [17011, 17348)...
[ 9743 ] Building cosine similarity matr

[ 9777 ] Building cosine similarity matrix for [17348, 17351)...
S done
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17351)
MAP@5: 0.08062957098695608
Iteration No: 921 ended. Search finished for the next optimal point.
Time taken: 85.9728
Function value obtained: -0.0806
Current minimum: -0.1051
Iteration No: 922 started. Searching for the next optimal point.
[0.30767378277190355, 0.16687641878876217, 0.20232091441395489, 0.8504719878260012, 0.075182589031791106, 0.89974714019807589]
[ 9781 ] Building cosine similarity matrix for [0, 1000)...
Running 4 workers...
[ 9782 ] Building cosine similarity matrix for [4337, 5337)...
[ 9783 ] Building cosine similarity matrix for [8674, 9674)...
[ 9784 ] Building cosine similarity matrix for [13011, 14011)...
[ 9781 ] Building cosine similarity matrix for [1000, 2000)...
[ 9782 ] Building cosine similarity matrix for [5337, 6337)...
[ 9783 ] Building cosine similarity matrix for [9674, 10674)...
[ 9784 ] Building cosine s

[ 9822 ] Building cosine similarity matrix for [14011, 15011)...
[ 9819 ] Building cosine similarity matrix for [2000, 3000)...
[ 9820 ] Building cosine similarity matrix for [6337, 7337)...
[ 9821 ] Building cosine similarity matrix for [10674, 11674)...
[ 9822 ] Building cosine similarity matrix for [15011, 16011)...
[ 9819 ] Building cosine similarity matrix for [3000, 4000)...
[ 9820 ] Building cosine similarity matrix for [7337, 8337)...
[ 9821 ] Building cosine similarity matrix for [11674, 12674)...
[ 9822 ] Building cosine similarity matrix for [16011, 17011)...
[ 9819 ] Building cosine similarity matrix for [4000, 4337)...
[ 9821 ] Building cosine similarity matrix for [12674, 13011)...
[ 9820 ] Building cosine similarity matrix for [8337, 8674)...
[ 9822 ] Building cosine similarity matrix for [17011, 17348)...
[ 9819 ] Building cosine similarity matrix for [17348, 17351)...
S done
Similarity matrix ready!
R_hat done
Shape of final matrix:  (4421, 17351)
MAP@5: 0.091881927165

Process ForkPoolWorker-3845:
Traceback (most recent call last):
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/pool.py", line 119, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/Cellar/python3/3.6.2/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "/Users/emanueleghelfi/Development/RS/recsys_challenge_2017/src/utils/matrix_utils.py", line 244, in _work_compute_cosine
    S_den = S_num.copy()
  File "/Users/emanueleghelfi/Development/RS/lib/python3.6/site-packages/scipy/spa

KeyboardInterrupt: 